In [1]:
##-----------------------------------------------------------------------------
##  Import
##-----------------------------------------------------------------------------
import numpy as np
from scipy import signal
import cv2
from PIL import Image, ImageDraw
from sobel import show_images
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import skimage.io as io
%matplotlib inline
%load_ext autoreload
%autoreload 2
##-----------------------------------------------------------------------------
##  Function
##-----------------------------------------------------------------------------
def searchInnerBound(img):


    Y = img.shape[0]
    X = img.shape[1]
    sect = X/4 		# Width of the external margin for which search is excluded
    minrad = 10
    maxrad = sect*0.8
    jump = 4 		# Precision of the coarse search, in pixels

    sz = np.array([np.floor(Y),
                    np.floor(X),
                    np.floor((maxrad-minrad))]).astype(int)
    integrationprecision = 1
    angs = np.arange(0, 2*np.pi, integrationprecision) 
    x, y, r = np.meshgrid(np.arange(sz[1]),
                          np.arange(sz[0]),
                          np.arange(sz[2]))
    
    y = sect + y*jump    #Y-points to be checked how many circles passes by them
    x = sect + x*jump   #X-Points to be checked how many circles passes by them
    r = minrad + r*jump #range of radiis
    hs = ContourIntegralCircular(img, y, x, r, angs)

    hspdr = hs - hs[:, :, np.insert(np.arange(hs.shape[2]-1), 0, 0)]
    sm = 3 		# Size of the blurring mask
    hspdrs = signal.fftconvolve(hspdr, np.ones([sm,sm,sm]), mode="same")

    indmax = np.argmax(hspdrs.ravel())
    y,x,r = np.unravel_index(indmax, hspdrs.shape)

    inner_y = sect + y*jump
    inner_x = sect + x*jump
    inner_r = minrad + (r-1)*jump


    integrationprecision = 0.1 		# Resolution of the circular integration
    angs = np.arange(0, 2*np.pi, integrationprecision)
    x, y, r = np.meshgrid(np.arange(jump*2),
                          np.arange(jump*2),
                          np.arange(jump*2))
    y = inner_y - jump + y
    x = inner_x - jump + x
    r = inner_r - jump + r
    
    hs = ContourIntegralCircular(img, y, x, r, angs)

    hspdr = hs - hs[:, :, np.insert(np.arange(hs.shape[2]-1), 0, 0)]

    sm = 2		# Size of the blurring mask
    hspdrs = signal.fftconvolve(hspdr, np.ones([sm,sm,sm]), mode="same")
    indmax = np.argmax(hspdrs.ravel())
    y,x,r = np.unravel_index(indmax, hspdrs.shape)

    inner_y = inner_y - jump + y
    inner_x = inner_x - jump + x
    inner_r = inner_r - jump + r - 1

    return inner_y, inner_x, inner_r


#------------------------------------------------------------------------------
def searchOuterBound(img, inner_y, inner_x, inner_r):

    maxdispl = np.round(inner_r*0.15).astype(int)

    minrad = np.round(inner_r/0.8).astype(int)
    maxrad = np.round(inner_r/0.3).astype(int)


    intreg = np.array([[2/6, 4/6], [8/6, 10/6]]) * np.pi

    integrationprecision = 0.05
    angs = np.concatenate([np.arange(intreg[0,0], intreg[0,1], integrationprecision),
                            np.arange(intreg[1,0], intreg[1,1], integrationprecision)],
                            axis=0)
    x, y, r = np.meshgrid(np.arange(2*maxdispl),
                          np.arange(2*maxdispl),
                          np.arange(maxrad-minrad))
    y = inner_y - maxdispl + y
    x = inner_x - maxdispl + x
    r = minrad + r
    hs = ContourIntegralCircular(img, y, x, r, angs)

    hspdr = hs - hs[:, :, np.insert(np.arange(hs.shape[2]-1), 0, 0)]

    sm = 7 	# Size of the blurring mask
    hspdrs = signal.fftconvolve(hspdr, np.ones([sm,sm,sm]), mode="same")

    indmax = np.argmax(hspdrs.ravel())
    y,x,r = np.unravel_index(indmax, hspdrs.shape)

    outer_y = inner_y - maxdispl + y + 1
    outer_x = inner_x - maxdispl + x + 1
    outer_r = minrad + r - 1

    return outer_y, outer_x, outer_r


#------------------------------------------------------------------------------
def ContourIntegralCircular(imagen, y_0, x_0, r, angs):

    print(len(angs), r.shape[0], r.shape[1], r.shape[2])
    y = np.zeros([len(angs), r.shape[0], r.shape[1], r.shape[2]], dtype=int)
    x = np.zeros([len(angs), r.shape[0], r.shape[1], r.shape[2]], dtype=int) 
    for i in range(len(angs)):
        ang = angs[i]
        y[i, :, :, :] = np.round(y_0 - np.cos(ang) * r).astype(int)
        x[i, :, :, :] = np.round(x_0 + np.sin(ang) * r).astype(int)
    
    ind = np.where(y < 0)
    y[ind] = 0
    ind = np.where(y >= imagen.shape[0])
    y[ind] = imagen.shape[0] - 1

    ind = np.where(x < 0)
    x[ind] = 0
    ind = np.where(x >= imagen.shape[1])
    x[ind] = imagen.shape[1] - 1


    hs = imagen[y, x]
    print("dsf ",hs.ndim)
    hs = np.sum(hs, axis=0)
    return hs.astype(float)
IMG_PATH = 'images/tanwnl3.bmp'
img = io.imread(IMG_PATH,0)
img = rgb2gray(img)
output_image = Image.new("RGB", Image.open(IMG_PATH).size)
output_image.paste(Image.open(IMG_PATH))
i_y,i_x,i_r = searchInnerBound(img)
draw_result = ImageDraw.Draw(output_image)
draw_result.ellipse((i_x-i_r, i_y-i_r, i_x+i_r, i_y+i_r), outline=(255,0,0))
o_y,o_x,o_r = searchOuterBound(img, i_y, i_x, i_r)
draw_result.ellipse((o_x-o_r, o_y-o_r, o_x+o_r, o_y+o_r), outline=(0,255,0))
output_image.save('images/ss.bmp')


ModuleNotFoundError: No module named 'scipy'